## Introduction
This notebook provides scripts to download Dominion game logs, convert them to state logs, and train a neural network model to predict the best card to buy.
The log files were too large to upload to github, but a zip files are provided for the state logs. Thus, it is optional to execute the first, third and fourth sections.

## 1. Download Logs
These cell define and execute the function to send an HTTP GET request to download the game log numbers specified.

In [ ]:
import requests

def getGameLog(lognum):
    logurl = "http://ceviri.me/woodcutter/" + lognum + "/plain/"
    fn = lognum + ".txt"

    logdata = requests.get(logurl)

    f = open(fn, "w")
    f.write(logdata)
    f.close()

In [ ]:
csvfile = open("DominionGameLogSpreadsheet.csv","r")
csvdata = csvfile.readlines()
csvfile.close()

startline = 1
endline = 10

for linenum in range(startline,endline+1):
    gameData = csvdata[linenum].split(',')
    gameNum = gameData[1]
    try:
        getGameLog(gameNum)
    except:
        print("Error with game" + gameNum)
        continue

## 2. Import Card Data
Run this cell to populate the required dictionaries. It loads each type of card and determines the number of inputs and outputs

In [ ]:
from array import array
import re
import csv
import numpy as np
import os
import tensorflow as tf

#load card data
cardFile = open("cards.txt","r")
cardData = cardFile.read().splitlines()
cardIndices = {}
cardOutputIndices = {}
cardTypes = {}
i = 0
n_possible_buys = 0
#cardOutputIndices.update({"nothing" : i})
for card in cardData:
    cardIndices.update({card: i})
    cardTypes.update({card: "card"})
    i+=3
    cardOutputIndices.update({card : n_possible_buys})
    n_possible_buys+=1
cardFile.close()

prizeFile = open("prizes.txt","r")
prizeData = prizeFile.read().splitlines()
for prize in prizeData:
    cardIndices.update({prize: i})
    cardTypes.update({prize: "prize"})
    i+=2
prizeFile.close()

eventFile = open("events.txt","r")
eventData = eventFile.read().splitlines()
for event in eventData:
    cardIndices.update({event: i})
    cardTypes.update({event: "event"})
    i+=1
    cardOutputIndices.update({event : n_possible_buys})
    n_possible_buys+=1
eventFile.close()

projectFile = open("projects.txt","r")
projectData = projectFile.read().splitlines()
for project in projectData:
    cardIndices.update({project: i})
    cardTypes.update({project: "project"})
    i+=3
    cardOutputIndices.update({project : n_possible_buys})
    n_possible_buys+=1
projectFile.close()

landmarkFile = open("landmarks.txt","r")
landmarkData = landmarkFile.read().splitlines()
for landmark in landmarkData:
    cardIndices.update({landmark: i})
    cardTypes.update({landmark: "landmark"})
    i+=1
landmarkFile.close()

numInputs = i #states, boons, and hexes are not inputs

stateFile = open("states.txt","r")
stateData = stateFile.read().splitlines()
for state in stateData:
    cardIndices.update({state: i})
    cardTypes.update({state: "state"})
    i+=1
stateFile.close()

boonHexFile = open("BoonsAndHexes.txt","r")
boonHexData = boonHexFile.read().splitlines()
for boonOrHex in boonHexData:
    cardIndices.update({boonOrHex: i})
    cardTypes.update({boonOrHex: "boonOrHex"})
    i+=1
boonHexFile.close()

artifactFile = open("artifacts.txt","r")
artifactData = artifactFile.read().splitlines()
for artifact in artifactData:
    cardIndices.update({artifact: i})
    cardTypes.update({artifact: "artifact"})
    i+=1
boonHexFile.close()

numElements = i



#print(cardIndices)
print("numInputs: " + str(numInputs))
print("numElements: " + str(numElements))
print("n_possible_buys: " + str(n_possible_buys))


## 3. Define Function to create State logs
This cell defines the function that converts the raw game logs to state logs that can be used as inputs to the neural network model. The logs to be converted should be placed in the "logs" directory.

In [ ]:
def MakeStateLogFromGameLog(gameNum,winner):
    fn = "logs/" + gameNum + ".txt"
    f = open(fn)
    fd = f.read()
    f.close()
    fLines = re.split('<br>',fd)

    supplyCards = re.split("[:,] ", fLines[1])
    #print(supplyCards)

    #initialize state array
    stateArray = np.zeros(numElements,np.uint8)

    #add cards from supply
    if re.match("Supply",supplyCards[0]):
        #print(1)
        #supplyCards = fLines[1].split(', ')
        #print(supplyCards)
        for entry in range(1,len(supplyCards)):
            try:
                match = re.match("(\d+) (\D+)",supplyCards[entry])
                num = np.uint8(match.group(1))
                card = match.group(2)
                cardIdx = cardIndices[card]
                cardType = cardTypes[card]
                if (cardType == "card"):
                    #add to supply
                    stateArray[cardIdx] += num

                elif ((cardType == "project") or (cardType == "event") or (cardType == "landmark")):
                    #indicate project, event, or landmark is present
                   stateArray[cardIdx] = 1

                elif ((cardType == "boonOrHex") or (cardType == "state") or (cardType == "artifact")):
                    continue #boons, hexes, states, and artifacts are not currently part of the input

                elif (cardType == "prize"):
                    continue #the presence of a prize is irrelevant since they are present if and only if Tournament is present
            except:
                print("ERROR: Unrecognized supply card " + supplyCards[entry] + " in game " + gameNum + ". State log was tossed.")
                statefile.close()
                os.remove(statefn)
                return

        statefn = "states/" + gameNum + ".txt"
        statefile = open(statefn,"w")
        
        #move starting cards into players' decks
        for j in range(2,len(fLines)):
            #split line into parts, e.g.: <TURN>:<PLAYER> - <COMMAND>: <QTY1> <Card1>, <QTY2> <Card2>
            #line = re.match("\d+:P(\d) \- (\w+)\W*((\d+) (\w+))?(, (\d+) (\w+))*",fLines[j])
            line = re.match("\d+:P(\d) \- \"?(\w+)\W*(\d+ \w+.*)",fLines[j])
            if line:
                playernum = int(line.group(1))
                player =  1 if (playernum == winner) else 2 #offset of 1 is winner,  offset of 2 is loser
                command = line.group(2)
                if ((player == 1) and (command == "BUY")):
                    #when the winner buys a card, save the state and the card bought. This is a set of inputs for training
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]

                                for k in range(num):
                                    statefile.write(card)
                                    for elem in range(numInputs):
                                        if stateArray[elem] > 200:
                                            #if a number is this large, we have underflow due to an error. Toss log and move on
                                            print("ERROR: Underflow detected in game " + gameNum + ". State log was tossed.")
                                            statefile.close()
                                            os.remove(statefn)
                                            return
                                        statefile.write("," + str(stateArray[elem]))
                                    statefile.write("\n")



                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                elif ((player == 1) and (command == "BUY_AND_GAIN")):
                    #when the winner buys a card, save the state and the card bought. This is a set of inputs for training
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]
                                gainType = cardTypes[card]

                                for k in range(num):
                                    statefile.write(card)
                                    for elem in range(numInputs):
                                        if stateArray[elem] > 200:
                                            #if a number is this large, we have underflow due to an error. Toss log and move on
                                            print("ERROR: Underflow detected in game " + gameNum + ". State log was tossed.")
                                            statefile.close()
                                            os.remove(statefn)
                                            return
                                        statefile.write("," + str(stateArray[elem]))
                                    statefile.write("\n")

                                    if (gainType == "card"):
                                        #move from supply to player's deck
                                        stateArray[gainIdx] -= 1
                                        stateArray[gainIdx + player] += 1

                                    elif (gainType == "project"):
                                        #update state of project for player
                                        stateArray[gainIdx + player] = 1

                                    elif (gainType == "event"):
                                        continue #events are not accumulated

                                    elif (gainType == "boonOrHex"):
                                        continue #boons and hexes are not currently part of the input

                                    elif (gainType == "state"):
                                        continue #states are not currently part of the input

                                    elif (gainType == "prize"):
                                        #update prize for player
                                        stateArray[gainIdx + player - 1] = 1



                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                elif ((command == "BUY_AND_GAIN") or (command == "GAIN") or (command == "\"GAIN") or (command == "RECEIVES") or (command == "RECEIVE") or (command == "BACK_ON_DECK")):
                    #add the card to the player's deck
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]
                                gainType = cardTypes[card]

                                if (gainType == "card"):
                                    #move from supply to player's deck
                                    stateArray[gainIdx] -= num
                                    stateArray[gainIdx + player] += num

                                elif (gainType == "project"):
                                    #update state of project for player
                                    stateArray[gainIdx + player] = 1

                                elif (gainType == "event"):
                                    continue #events are not accumulated

                                elif (gainType == "boonOrHex"):
                                    continue #boons and hexes are not currently part of the input

                                elif (gainType == "state"):
                                    continue #states are not currently part of the input

                                elif (gainType == "prize"):
                                    #update prize for player
                                    stateArray[gainIdx + player - 1] = 1



                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                elif ((command == "BUY")):
                    #this is a buy without "gaining" anything
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]
                                gainType = cardTypes[card]

                                if (gainType == "project"):
                                    #update state of project for player
                                    stateArray[gainIdx + player] = 1

                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return



                elif ((command == "INSERT_IN_DECK") or (command == "STARTS_WITH")):
                    #move from supply to player's deck
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]

                                stateArray[gainIdx] -= num
                                stateArray[gainIdx + player] += num

                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                elif (command == "TRASH"):
                    #move from player's deck to trash
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                #print(match)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]

                                stateArray[gainIdx + player] -= num
                                #print(stateArray[gainIdx + player])

                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                elif ((command == "RETURN") or (command == "DECK_TO_DISCARD")):
                    #move from player's deck to supply
                    if line.group(3):
                        cardGroups = line.group(3).split(", ")
                        for group in cardGroups:
                            try:
                                match = re.match("(\d+) (\D+)",group)
                                #print(match)
                                num = np.uint8(match.group(1))
                                card = match.group(2)
                                gainIdx = cardIndices[card]

                                cardType = cardTypes[card]
                                if (cardType == "card"):
                                    stateArray[gainIdx + player] -= num
                                    stateArray[gainIdx] += num

                                elif ((cardType == "boonOrHex") or (cardType == "state")):
                                    continue #boons, hexes, and states are not currently part of the input

                            except:
                                print("Error in " + command + " in game " + gameNum + ". State log was tossed.")
                                statefile.close()
                                os.remove(statefn)
                                return

                
                # elif ((command == "DRAW") or (command == "PLAY") or (command == "COUNTER_ADD") or (command == "PLAYER_FLIPS_TOKEN_UP") or (command == "PLAYER_FLIPS_TOKEN_DOWN")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "DISCARD") or (command == "EXCHANGE_RETURN") or (command == "REVEAL_FINDING") or (command == "REVEAL_SEARCHING_AND_FINDING") or (command == "GETS_BUYS")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "REVEAL") or (command == "BOTTOMDECKS") or (command == "TOPDECK") or (command == "GETS_ACTION") or (command == "GETS_BUY")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "OBELISK_FAILS") or (command == "USES_VILLAGER") or (command == "GETS_VILLAGERS_FROM") or (command == "RECEIVES_GENERIC")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "TAKE") or (command == "REVEAL_SEARCHING") or (command == "PUT_IN_HAND") or (command == "DRAW_FROM_HAUNTED_WOODS") or (command == "LOOK_AT")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "PUT_IN_HAND_FROM_CRYPT") or (command == "SET_ASIDE") or (command == "REVEAL_HAND") or (command == "PLAY_AGAIN")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "PUT_ON_MAT") or (command == "EXPLAIN_POINTS_PASTURE") or (command == "DRAW_FROM_HIRELING") or (command == "GETS_COINS")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "TACTICIAN_FAIL") or (command == "PLAY_THIRD") or (command == "SHUFFLE_INTO_DECK") or (command == "EXPLAIN_POINTS_FAIRGROUNDS")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "DISCARD_RECEIVABLE") or (command == "RETURNS_TOKEN") or (command == "EXPLAIN_POINTS_DUH") or (command == "EXPLAIN_POINTS_DUKE")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "CALL") or (command == "GETS_COIN") or (command == "COINS_FROM_SECRET_CAVE") or (command == "SET_ASIDE_WITH")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "DRAW_FROM_ENCHANTRESS") or (command == "DRAW_FROM_CARAVAN") or (command == "REPAYS_DEBT") or (command == "NAMES")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "DRAW_FROM_WHARF") or (command == "EXPLAIN_POINTS_VP_TOKENS") or (command == "DRAW_FROM_GHOST_TOWN") or (command == "THE_SKYS_GIFT_FAIL")):
                #     #these can be skipped since they don't produce useful info
                #     continue
                # elif ((command == "PRIEST_ACTIVATES") or (command == "LOSES_COIN") or (command == "GETS_COIN_FROM") or (command == "COINS_FROM_RAIDER")):
                #     continue
                # elif ((command == "TAX_ADD_DEBT") or (command == "ADVANCE_FAIL") or (command == "INHERITS")):
                #     #these can be skipped since they don't produce useful info
                #     continue

                
                # else:
                #     print("Unrecognized command " + command + " in game " + gameNum)
                #     print(line)
                #     continue
               

            else:
                continue

        statefile.close()
        print("Wrote state log for game " + gameNum + " successfully")
    else:
        print("Error with file for game " + gameNum)



## 4. Convert game logs to state logs
Run this cell to convert a batch of game logs to state logs. It gets each game log number and winner from the CSV file, then uses those as inputs to the function defined above. If an error is encountered, the log is skipped.

In [ ]:
csvfile = open("DominionGameLogSpreadsheet.csv","r")
csvdata = csvfile.readlines()
csvfile.close()

startline = 100
endline = 100000

for linenum in range(startline,endline+1):
    gameData = csvdata[linenum].split(',')
    gameNum = gameData[1]
    P1name = gameData[2]
    P2name = gameData[4]
    winnerName = gameData[6]
    if P1name == winnerName:
        winner = 1
    elif P2name == winnerName:
        winner = 2
    else:
        print("Something was wrong with the file. Skipping game " + gameNum)
        continue
    try:
        MakeStateLogFromGameLog(gameNum,winner)
    except:
        print("Error with game" + gameNum)
        continue


## 5. Load state log data for training
This cell loads the data from the state log files so it can be input into the neural network model. The state logs should be placed in the "states" directory.

In [ ]:
statelogs = os.listdir('states') 
for elem in range(len(statelogs)):
    statelogs[elem] = "states/" + statelogs[elem]
len(statelogs)

#take subset for testing code
NUM_SETS_TO_USE = 5000
if NUM_SETS_TO_USE > len(statelogs):
    NUM_SETS_TO_USE = len(statelogs)

statelogs = statelogs[:NUM_SETS_TO_USE]

#initialize training and testing arrays
x_train = []
y_train = []
x_test = []
y_test = []

n_training_set = round(0.75*len(statelogs))

statefilename = statelogs[1]
statefile = open(statefilename,"r")
statedata = statefile.read().splitlines()

for statefileidx in range(n_training_set):
    statefile = open(statelogs[statefileidx],"r")
    statedata = statefile.read().splitlines()

    for line in statedata:
        linearr = line.split(',')
        y = cardOutputIndices[linearr[0]]
        #y = linearr[0]
        #x = np.array(list(map(int, linearr[1:])))
        #x = np.array(map(float, linearr[1:]))
        #x = np.array([float(s) for s in linearr[1:]])
        x = tuple(map(int,linearr[1:]))
        if(len(x) == numInputs):
            y_train.append(y)
            x_train.append(x)
        else:
            print("ERROR: x was length " + str(len(x)))
        #y_train = np.append(y_train,y)
        #x_train = np.append(x_train,x)

for statefileidx in range(n_training_set,len(statelogs)):
    statefile = open(statelogs[statefileidx],"r")
    statedata = statefile.read().splitlines()

    for line in statedata:
        linearr = line.split(',')
        y = cardOutputIndices[linearr[0]]
        #y = linearr[0]
        #x = np.array(list(map(int, linearr[1:])))
        #x = np.array(map(float, linearr[1:]))
        #x = np.array([float(s) for s in linearr[1:]])
        x = tuple(map(int,linearr[1:]))
        if(len(x) == numInputs):
            y_test.append(y)
            x_test.append(x)
        else:
            print("ERROR: x was length " + str(len(x)))
        #y_test = np.append(y_test,y)
        #x_test = np.append(x_test,x)


#x_train = np.array(x_train)
#x_test = np.array(x_test)
#y_train = np.array(y_train)
#y_test = np.array(y_test)

try:
    #convert to numpy array
    #x_train = np.asarray(x_train).astype('float32')
    #x_test = np.asarray(x_test).astype('float32')
    x_train = np.asarray(x_train).astype('uint8')
    x_test = np.asarray(x_test).astype('uint8')
    y_train = np.asarray(y_train).astype('int16')
    y_test = np.asarray(y_test).astype('int16')

    #x_train, x_test = x_train / 255.0, x_test / 255.0
except:
    print("ERROR:")
    print(x_test.shape)
    for elem in x_test:
        print(x_test.shape)


## 6. Train
Run this cell to train and evaluate a neural network on the data loaded above.

In [ ]:
#model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
 # tf.keras.layers.Flatten(input_shape=(numInputs,1)),
 # tf.keras.layers.Dense(2048, activation='relu', input_shape=(numInputs,)),
 # tf.keras.layers.Dense(2048, activation='relu'),
 # tf.keras.layers.Dense(1024, activation='sigmoid'),
 # tf.keras.layers.Dropout(0.2),
 # tf.keras.layers.Dense(n_possible_buys)
#])

inputs = tf.keras.Input(shape=(numInputs,))
L1 = tf.keras.layers.Dense(4096, activation=tf.nn.relu)(inputs)
L2 = tf.keras.layers.Dense(1024, activation=tf.nn.sigmoid)(L1)
outputs = tf.keras.layers.Dense(n_possible_buys, activation=tf.nn.softmax)(L2)
#outputs = tf.keras.layers.Dense(n_possible_buys, activation=tf.nn.sigmoid)(L2)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

predictions = model(x_train[:1]).numpy()
#predictions = model(x_train[0]).numpy()

#predictions = model(x_train[0]).numpy()

#print("predictions: \n", predictions)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

print("loss: \n", loss_fn(y_train[:1], predictions).numpy())
#print("loss: \n", loss_fn(y_train[:1], predictions))
#print("loss: \n", loss_fn(y_train[:1], predictions))
#print("loss: \n", loss_fn(y_train, predictions).numpy())

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
              
model.fit(x_train, y_train, epochs=5, batch_size=32)
#model.fit(x_train, y_train, epochs=5, batch_size=32)

print("evaluation: \n")

model.evaluate(x_test,  y_test, verbose=1)

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

#print("probabilities: \n", probability_model(x_test[:5]))
print("probabilities: \n", model(x_test[:5]))

## 7. Analysis
Run this cell to analyze the top outputs for a specified range of inputs. It prints the top 3 cards from each, along with their probabilities.

In [ ]:
#print(max(probability_model(x_test[:5])).all())
#np.argsort(probability_model(x_test[:5]))
#for row in probability_model(x_test[:5]):
#    np.argsort()
#    print(float(max(row)))
#print(probability_model(x_test[:5])[1][37])
start_idx = 135
end_idx = 140
pm = probability_model(x_test[start_idx:end_idx])

for line in pm:
    top3indices = np.flip(np.argsort(line)[-3:])
    cardList = list(cardOutputIndices)
    print(top3indices)
    for index in top3indices:
        print(cardList[index],float(line[index]))
        #print(line[index])

## Testing/Debugging
The cells below were to assist in debugging

In [ ]:
#import tensorflow as tf
#dataset = tf.data.TextLineDataset(["file1.txt", "file2.txt"])

#testdata = tf.data.TextLineDataset(statelogs[1:10])
#print(testdata.cardinality())
#for elem in testdata:
#    print(elem)

#for x in range(10):
#    print(y_train[x])

#mnist = tf.keras.datasets.mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train, x_test = x_train / 255.0, x_test / 255.0

print(len(x_train))
print(len(x_train[0]))
#print(len(x_train[0][0]))
print(len(x_train[:1]))
print(x_train[:1])
print((x_train[:1])[:1])

print(len(y_train))
print(y_train[0])
print(y_train)

print(x_train.shape)

print(x_train[:1].shape)

print(y_train.shape)
print(y_train[:1].shape)

In [ ]:
print(x_train[1])

In [ ]:
print(model.input_shape)


In [ ]:
print(list(cardOutputIndices)[1])

In [ ]:
model.summary()
print(x_test.shape)
print(x_test[0].shape)
print(y_train[:1].shape)
print(y_train.shape)
print(y_train[0].shape)
print(y_train[:1])

## AND Gate model
The following two cells provide an AND gate example that was done to ensure the input format was correct

In [ ]:
import os
import tensorflow as tf
import numpy as np


#take subset for testing code
NUM_SETS_TO_USE = 500

#initialize training and testing set lists
y_train = []
x_train = []
y_test = []
x_test = []

statefilename = "test.txt"
statefile = open(statefilename,"r")
statedata = statefile.read().splitlines()

txt2num = {}

txt2num.update({"True": 1})
txt2num.update({"False": 2})

for statefileidx in range(NUM_SETS_TO_USE):

    for line in statedata:
        linearr = line.split(',')
        y = txt2num[linearr[0]]
        #y = linearr[0]
        x = np.array(list(map(int, linearr[1:])))
        y_train.append(y)
        x_train.append(x)

for statefileidx in range(round(NUM_SETS_TO_USE/2)):

    for line in statedata:
        linearr = line.split(',')
        y = txt2num[linearr[0]]
        #y = linearr[0]
        x = np.array(list(map(int, linearr[1:])))
        y_test.append(y)
        x_test.append(x)


x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

#x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
#model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
  #tf.keras.layers.Flatten(input_shape=(2,1)),
 # tf.keras.Input(shape=(2,)),
  #tf.keras.layers.Dense(128, activation='sigmoid', input_shape=(2,)),
  #tf.keras.layers.Dense(1024, activation='relu'),
  #tf.keras.layers.Dense(128, activation='sigmoid'),
  #tf.keras.layers.Dropout(0.2),
  #tf.keras.layers.Dense(2)
#])

inputs = tf.keras.Input(shape=(2,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(3, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

predictions = model(x_train[:1]).numpy()

#predictions = model(x_train[0]).numpy()

print("predictions: \n", predictions)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print("loss: \n", loss_fn(y_train[:1], predictions).numpy())
#print("loss: \n", loss_fn(y_train[:1], predictions))
#print("loss: \n", loss_fn(y_train, predictions).numpy())

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
              
model.fit(x_train, y_train, epochs=5)

print("evaluation: \n")

model.evaluate(x_test,  y_test, verbose=1)

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

print("probabilities: \n", probability_model(x_test[:5]))